In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import logging
import pandas as pd
import numpy as np
sys.path.append(os.path.dirname(os.path.abspath('')))
from auto_ab.splitter.params import SplitBuilderParams

logging.basicConfig(level = logging.INFO)

%load_ext autoreload
%autoreload 2
from auto_ab.splitter.split_builder import SplitBuilder

# Кейс №6. Выделение групп

In [2]:
df = pd.read_csv('./data/ab_data.csv')
df["moda_city"] = np.random.randint(1, 5, df.shape[0])
df["moda_city"] = df["moda_city"].astype(str)
df["country"] = np.random.randint(1, 3, df.shape[0])
df["id"] = df.index

In [3]:
split_builder_params = SplitBuilderParams(
    map_group_names_to_sizes={
        'control': 10000,
        'target': 20000,
        'target1': 30000
    },
    main_strata_col = "moda_city",
    split_metric_col = "height_now",
    id_col = "id",
    cols = ["height_prev"],
    cat_cols=["country"],
    pvalue=0.05,
    n_top_cat=5,
    stat_test="ttest_ind",
    n_bins_rto = 6,
    bin_min_size = 500
)

In [4]:
split_builder = SplitBuilder(df, split_builder_params)

In [5]:
split = split_builder.collect()

In [6]:
split.head()

,height_now,height_prev,weight_now,weight_prev,noise_now,noise_prev,groups,id,moda_city,country,numerator,denominator,conversion,strata,group_name
0,178.610565,173.386388,173.195935,180.831845,11.664751,2.711829,B,1,2,0.50103,1,4,1,22-1,target
1,167.590042,167.684244,166.109779,170.189261,9.134517,-1.153977,B,4,3,0.50103,3,2,1,30-1,target
2,171.061403,166.374731,167.835024,172.793484,4.044180,7.901575,B,9,1,0.49897,2,2,1,101,control
3,184.111529,182.478653,179.940720,186.576711,1.426471,0.685937,B,10,4,0.49897,2,2,1,42-1,control
4,167.449021,163.500754,161.494712,169.214959,10.096040,6.560057,A,17,1,0.50103,2,4,1,10-1,control
